In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

****TEST SCORE

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn import preprocessing
from statsmodels.stats.diagnostic import het_white

In [ ]:
df_test_score  = pd.read_csv('/kaggle/input/predict-test-scores-of-students/test_scores.csv')

In [ ]:
df_test_score.head()

In [ ]:
df_test_score.info()

In [ ]:
df_test_score.describe()

We drop the id since it has no role in the predicting task at all.

In [ ]:
df_test_score = df_test_score.drop(columns=['student_id'])

The data is transformed into categorical.

In [ ]:
for col in ['school' ,'school_setting', 'school_type', 'classroom', 'teaching_method', 'gender', 'lunch']:
    df_test_score[col] = df_test_score[col].astype('category')

We make sure there is no null values.

In [ ]:
df_test_score.isnull().sum()

****EDA

We start the exploration of the numerical variables.

In [ ]:
df_test_score.hist()
plt.show()

More interesting than the single univariate plots, we can explore relationship betewwen each pair of variables. immediately it outcomes a linear relathionship between de variables pretest and posttest. By the other hand, it seems to exists a different behaviour in extreme regions of n_students where variable is too high or to low.

In [ ]:
attributes = ['n_student', 'pretest', 'posttest']
scatter_matrix(df_test_score[attributes])

How much variance is explained by pretest? The answer is below

In [ ]:
lin_reg =  LinearRegression()
x = np.array(df_test_score['pretest']).reshape(-1, 1)
y = np.array(df_test_score['posttest']).reshape(-1, 1)
lin_reg.fit(x, y)

In [ ]:
r_sq = lin_reg.score(x, y)
print('coefficient of determination:', r_sq)

A 90% of the variance is already explained by one variable. So, the strategy consists of including the most relevant variables and eliminating the noising ones. We also explore the categorical-response relationship.

In [ ]:
for col in ['school' , 'school_setting', 'school_type', 'classroom', 'teaching_method', 'gender', 'lunch']:
  sns.catplot(x=col, y="posttest", kind="box", data=df_test_score)

****Variable Selection

We are going to use a base tree model to select the variables, i.e. random forest. We proved two more models (XGboost and CART) with the same variable selection.

Let's use numerical encoding for the categorical features.

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(df_test_score['gender'])
le.transform(df_test_score['gender'])
df_train = df_test_score
for col in ['school' , 'school_setting', 'school_type', 'classroom', 'teaching_method', 'gender', 'lunch']:
  le.fit(df_train[col])
  df_train[col] = le.transform(df_train[col])

In [ ]:
X = df_train.drop(columns=['posttest'])
y = df_train['posttest']
dt_model = RandomForestRegressor()
dt_model.fit(X, y)

In [ ]:
feature_list = list(X.columns)
# Set the style
importances = list(dt_model.feature_importances_)
plt.style.use('fivethirtyeight')
x_values = list(range(len(importances)))
plt.bar(x_values, importances, orientation = 'vertical')
plt.xticks(x_values, feature_list, rotation='vertical')
plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances');

We select only those variables that explain the 95% of the variance. As a result, only two variables are selected.

In [ ]:
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

sorted_importances = [importance[1] for importance in feature_importances]
sorted_features = [importance[0] for importance in feature_importances]
# Cumulative importances
cumulative_importances = np.cumsum(sorted_importances)
# Make a line graph
plt.plot(x_values, cumulative_importances, 'g-')
# Draw line at 95% of importance retained
plt.hlines(y = 0.95, xmin=0, xmax=len(sorted_importances), color = 'r', linestyles = 'dashed')
# Format x ticks and labels
plt.xticks(x_values, sorted_features, rotation = 'vertical')
# Axis labels and title
plt.xlabel('Variable'); plt.ylabel('Cumulative Importance'); plt.title('Cumulative Importances');

In [ ]:
df_sel_var = df_test_score[['pretest', 'teaching_method', 'posttest']]

****Model Training

Finally, we train a simple linear regression model.

In [ ]:
X = df_sel_var.drop(columns=['posttest'])
y = df_sel_var['posttest']
X_training_data, X_testing_data, y_training_data, y_testing_data = train_test_split(X, y, test_size=0.3)

In [ ]:
final_model =  LinearRegression()
final_model.fit(X_training_data, y_training_data)

In [ ]:
r_sq = final_model.score(X_training_data, y_training_data)
print('Coefficient of determination for training data:', r_sq)

In [ ]:
X1 = sm.add_constant(X_training_data)
modelOLS = sm.OLS(y_training_data, X1)
modelOLS_fit = modelOLS.fit()
modelOLS_fit.summary()

The Jarque-Bera test ensures the normality of the residuals and all the coeficients are significant and the rest of hipothesis underliying linear model, except ot Heteroskedasticity.

In [ ]:
white_test = het_white(modelOLS_fit.resid,  modelOLS_fit.model.exog)

In [ ]:
#white_test
labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']
print(dict(zip(labels, white_test)))

Heteroskedasticity assumption is also checked.

In [ ]:
y_pred = final_model.predict(X_testing_data)
print('Validation MAE', mean_absolute_error(y_testing_data, y_pred))
print('Validation RMSE', sqrt(mean_squared_error(y_testing_data, y_pred)))
print('Validation R^2', final_model.score(X_testing_data, y_testing_data))

The predictions are in generral quiet accurate.